In [6]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, RobustScaler , MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from azureml.core import Workspace, Experiment
from math import sqrt
from utils import get_grouped_df
import pandas as pd
import joblib
import numpy as np

def preprocess_data(df):
    # Séparation des listes
    df['nom_acteur'] = df['nom_acteur'].str.split(', ')
    df['nom_compagnie'] = df['nom_compagnie'].str.split(', ')
    df['nom_genre'] = df['nom_genre'].str.split(', ')
    
    # Binarizers
    mlb_actor = MultiLabelBinarizer()
    mlb_company = MultiLabelBinarizer()
    mlb_genre = MultiLabelBinarizer()
    
    # Transformations
# Transformer la colonne des acteurs
    df = df.join(pd.DataFrame(mlb_actor.fit_transform(df.pop('nom_acteur')),
                            columns=[f'actor_{cls}' for cls in mlb_actor.classes_],
                            index=df.index))

    # Transformer la colonne des compagnies
    df = df.join(pd.DataFrame(mlb_company.fit_transform(df.pop('nom_compagnie')),
                            columns=[f'company_{cls}' for cls in mlb_company.classes_],
                            index=df.index))

    # Transformer la colonne des genres
    df = df.join(pd.DataFrame(mlb_genre.fit_transform(df.pop('nom_genre')),
                            columns=[f'genre_{cls}' for cls in mlb_genre.classes_],
                            index=df.index))

    # Sauvegarde des transformations
    joblib.dump(mlb_actor, 'mlb_actor.pkl')
    joblib.dump(mlb_company, 'mlb_company.pkl')
    joblib.dump(mlb_genre, 'mlb_genre.pkl')

    return df.fillna(0)

# Charger les données
df = get_grouped_df()

# Sauvegarde du titre pour plus tard
df_title = df['title'].copy()

# Prétraitement
df_t = preprocess_data(df)

df_t


/home/apprenant/Desktop/Predictif_des_Entrees_de_Films/notebooks/utils.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,title,entries,date,realisateur,budget,titre_non_modifie,film_id,actor_50 Cent,actor_A$AP Rocky,actor_A.J. Cook,...,genre_Fantastique,genre_Guerre,genre_Histoire,genre_Horreur,genre_Musique,genre_Mystère,genre_Romance,genre_Science-Fiction,genre_Thriller,genre_Western
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2741,zerotheorem,40523,2014-06-25,Terry Gilliam,8500000,Zero Theorem,1779,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2742,zookeeper,73107,2011-08-17,Frank Coraci,80000000,Zookeeper,576,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2743,zoolander2,46701,2016-03-02,Ben Stiller,50000000,Zoolander 2,1223,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2744,zootopie,1556396,2016-02-17,Byron Howard,150000000,Zootopie,1931,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def preprocess_inputs(df):
    df = df.drop(['title', 'date', 'titre_non_modifie', 'film_id'], axis=1)
    y = df['entries']
    X = df.drop('entries', axis=1)
    return X, y

def build_model():
    # Transformation polynomiale
    poly_transformer = Pipeline(steps=[
        ('scaler', RobustScaler()),
        ('poly', PolynomialFeatures(degree=2))
    ])

    # Transformation OneHot
    cat_transformer = OneHotEncoder(handle_unknown='ignore')

    # Assemblage des transformations
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', poly_transformer, ['budget']),
            ('cat', cat_transformer, ['realisateur'])],
        remainder='passthrough')

    # Pipeline RandomForestRegressor
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestRegressor(random_state=42))
    ])
    return model

# Prétraitement des entrées
X, y = preprocess_inputs(df_t)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construction du modèle
model = build_model()

# Formation du modèle
model.fit(X_train, y_train)

# Prédiction
y_pred = model.predict(X_test).astype(int)

# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = sqrt(mse)

print(f"MSE: {mse}, MAE: {mae}, R2: {r2}, RMSE: {rmse}")


# Sauvegarder le modèle dans un fichier .pkl
model_filename = 'model.pkl'
joblib.dump(model, model_filename)

# Charger le modèle à partir du fichier .pkl
loaded_model = joblib.load(model_filename)


MSE: 85098555353.23817, MAE: 164478.74727272728, R2: 0.5225659159527098, RMSE: 291716.566813128


In [15]:
# Créer un DataFrame pour les prédictions et les valeurs réelles
df_results = pd.DataFrame({'Title': df_title.loc[X_test.index], 'Actual': y_test, 'Predicted': y_pred})
df_results


,Title,Actual,Predicted
1264,lavenir,102832,32504
2359,takinglivesdestinsvioles,224246,165044
1778,michaelclayton,254575,151595
1632,lincoln,437768,344382
2657,vickycristinabarcelona,565395,324318
...,...,...,...
1775,miaetlelionblanc,573356,317335
2260,skyscraper,239014,421068
179,arretemoisitupeux,1137654,301277
1738,mary,55647,47082


In [17]:
# Données d'entrée pour la prédiction
data = {
    "titre_fr": ["spidermanfarfromhome"],
    "realisateur": ["Jon Watts"],
    "acteurs": [["Tom Holland", "Samuel L. Jackson", "Jake Gyllenhaal"]],
    "genres": [["Action", "Aventure", "Science-Fiction"]],
    "budget": [160000000],
    "date_sortie": ["2019-07-03"],
    "compagnies_production": [["Marvel Studios", "Pascal Pictures"]],
    "titre_non_modifie": ["Spider-Man : Far From Home"]
}

# Créer un DataFrame à partir des données
df_input = pd.DataFrame(data)

# Prétraitement des données pour les nouvelles fonctionnalités
df_input['date_sortie'] = pd.to_datetime(df_input['date_sortie'])
df_input['year'] = df_input['date_sortie'].dt.year
df_input['month'] = df_input['date_sortie'].dt.month
df_input['day'] = df_input['date_sortie'].dt.day

# Supprimer les colonnes non utilisées pour la prédiction
df_input = df_input.drop(['titre_fr', 'date_sortie'], axis=1)

# Charger les transformations des catégories
mlb_actor = joblib.load('mlb_actor.pkl')
mlb_company = joblib.load('mlb_company.pkl')
mlb_genre = joblib.load('mlb_genre.pkl')

# Transformer les listes d'acteurs, de compagnies et de genres en plusieurs colonnes binaires
df_input = df_input.join(pd.DataFrame(mlb_actor.transform(df_input.pop('acteurs')),
                                      columns=[f'actor_{cls}' for cls in mlb_actor.classes_],
                                      index=df_input.index))

df_input = df_input.join(pd.DataFrame(mlb_company.transform(df_input.pop('compagnies_production')),
                                      columns=[f'company_{cls}' for cls in mlb_company.classes_],
                                      index=df_input.index))

df_input = df_input.join(pd.DataFrame(mlb_genre.transform(df_input.pop('genres')),
                                      columns=[f'genre_{cls}' for cls in mlb_genre.classes_],
                                      index=df_input.index))

# Prédiction avec le modèle
prediction = loaded_model.predict(df_input).astype(int)

# Afficher la prédiction
df_results = pd.DataFrame({'Title': data['titre_non_modifie'], 'Actual': prediction})
print(df_results)

                        Title  Actual
0  Spider-Man : Far From Home  974337
